In [2]:
#### Setting up Spark for colab space, code provided by Professor Othman

import os
#Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Set JAVA_HOME path variable in Linux
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version


#Install Spark
#download Spark file
!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#extract the file
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#Set SPARK-HOME path variable in Linux
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"


#install findspark package
!pip install -q findspark

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode
openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)


In [3]:
import findspark
findspark.init()

In [4]:
# create entry points to spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
conf = SparkConf().setAppName("lecture10").setMaster("local[*]")
sc=SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)
sqlContext = SQLContext(sc)

In [5]:
spark

In [118]:
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import substring, lit, desc, col
import pyspark.sql.functions as F

In [7]:
# hardcoded variables telling our program which states, years, and tables are needed from NIBRS dataa

# Can make the list longer if running locally, for google colab, better to do one state at a time

state_list = ["alabama", "arizona", "arkansas","colorado", "connecticut"]
state_abbr = ["AL", "AZ", "AR", "CO", "CT"]
csv = ["nibrs_circumstances", "nibrs_victim_circumstances", "nibrs_victim", "ref_race"]
firstyear = 1991
lastyear = 2018

In [34]:
def read_nibrs_csv(firstyear,lastyear,state_name,state_abbr, csv, my_schema):
    url_base="https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/"

    df_list = []
    for year in range(firstyear,lastyear+1):
        url = url_base+state_name+'/'+state_abbr+'-'+str(year)
        url_c = url + '/' + csv + '.csv'
              
        try:
            temp_df = pd.read_csv(url_c, error_bad_lines=False)
        except:
            # there was a format change in 2016 where NIBRS_data became capitalized
            url_c = url + '/' + csv.upper() + '.csv' 
            try:
                temp_df = pd.read_csv(url_c, error_bad_lines=False)
                temp_df.columns = map(str.lower, temp_df.columns)
            except:
                print(url_c + " did not exist")
              
        temp_df['state'] = state_abbr
        temp_df['year'] = year
        df_list.append(temp_df)

    full_df = pd.concat(df_list)
    spark_df = spark.createDataFrame(full_df,schema=my_schema)

    return spark_df
    #return full_df
            


In [31]:
arrestee_AL

,arrestee_id,incident_id,arrestee_seq_num,arrest_num,arrest_date,arrest_type_id,multiple_indicator,offense_type_id,age_id,age_num,sex_code,race_id,ethnicity_id,resident_code,under_18_disposition_code,clearance_ind,ff_line_number,age_range_low_num,age_range_high_num,state,year,data_year
0,1614,129882,1,L55P5MOKP,1991-07-20 00:00:00,1,N,27,5,23.0,F,2,3,R,NaN,NaN,NaN,NaN,NaN,AL,1991,NaN
1,1615,130130,1,5TPPOPMKP,1991-10-16 00:00:00,1,N,49,5,19.0,M,2,3,R,NaN,NaN,NaN,NaN,NaN,AL,1991,NaN
2,1650,130935,1,KPMP95MM5,1991-01-28 00:00:00,3,N,40,5,37.0,F,2,3,N,NaN,NaN,NaN,NaN,NaN,AL,1991,NaN
3,1649,130924,1,KPMP9TMPM,1991-01-25 00:00:00,1,N,23,5,14.0,M,1,3,R,R,NaN,NaN,NaN,NaN,AL,1991,NaN
4,1648,130912,1,KPMP9LMPZ,1991-01-25 00:00:00,1,N,23,5,37.0,M,2,3,R,NaN,NaN,NaN,NaN,NaN,AL,1991,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,27999562,95483372,1,NaN,24-JAN-18,1,N,49,5,46.0,M,2,2,N,NaN,NaN,NaN,46.0,0.0,AL,2018,2018.0
136,27999575,95483418,1,NaN,28-JAN-18,1,N,16,5,28.0,M,2,2,N,NaN,NaN,NaN,28.0,0.0,AL,2018,2018.0
137,27999532,95483274,3,NaN,10-JAN-18,1,N,39,5,18.0,F,1,2,N,NaN,NaN,NaN,18.0,0.0,AL,2018,2018.0
138,27999529,95483274,1,NaN,10-JAN-18,1,N,39,5,19.0,M,2,2,N,NaN,NaN,NaN,19.0,0.0,AL,2018,2018.0


In [23]:
circumstances_schema = StructType([ StructField("circumstances_id", IntegerType(), True)\

                       ,StructField("circumstances_type", StringType(), True)\

                       ,StructField("circumstances_code", IntegerType(), True)\

                       ,StructField("circumstances_name", StringType(), True)\

                       ,StructField("state", StringType(), True)\
                       
                       ,StructField("year", IntegerType(), True)])

In [53]:
arrestee_schema = StructType([ StructField("arrestee_id", IntegerType(), True)\

                       ,StructField("incident_id", IntegerType(), True)\

                       ,StructField("arrestee_seq_num", IntegerType(), True)\

                       ,StructField("arrest_num", StringType(), True)\

                       ,StructField("arrest_date", StringType(), True)\
                       
                       ,StructField("arrest_type_id", IntegerType(), True)\
                       
                       ,StructField("multiple_indicator", StringType(), True)\
                       
                       ,StructField("offense_type_id", IntegerType(), True)\
                       
                       ,StructField("age_id", IntegerType(), True)\
                       
                       ,StructField("age_num", DoubleType(), True)\
                       
                       ,StructField("sex_code", StringType(), True)\
                       
                       ,StructField("race_id", IntegerType(), True)\
                       
                       ,StructField("ethnicity_id", IntegerType(), True)\
                       
                       ,StructField("resident_code", StringType(), True)\
                       
                       ,StructField("under_18_disposition_code", StringType(), True)\
                       
                       ,StructField("clearance_ind", DoubleType(), True)\
                       
                       ,StructField("ff_line_number", DoubleType(), True)\
                       
                       ,StructField("age_range_low_num", DoubleType(), True)\
                       
                       ,StructField("age_range_high_num", DoubleType(), True)\
                       
                       ,StructField("state", StringType(), True)\
                       
                       ,StructField("year", IntegerType(), True)\
                       
                       ,StructField("data_year", DoubleType(), True)])

In [72]:
arrestee_AL = read_nibrs_csv(1991,2018,'alabama','AL','nibrs_arrestee', arrestee_schema)

https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1993/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1994/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1995/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1996/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1997/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1998/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-1999/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched1atwit/CS3800_4050/master/data/alabama/AL-2000/NIBRS_ARRESTEE.csv did not exist
https://raw.githubusercontent.com/roched

In [73]:
arrestee_AL.show()

+-----------+-----------+----------------+------------+-------------------+--------------+------------------+---------------+------+-------+--------+-------+------------+-------------+-------------------------+-------------+--------------+-----------------+------------------+-----+----+---------+
|arrestee_id|incident_id|arrestee_seq_num|  arrest_num|        arrest_date|arrest_type_id|multiple_indicator|offense_type_id|age_id|age_num|sex_code|race_id|ethnicity_id|resident_code|under_18_disposition_code|clearance_ind|ff_line_number|age_range_low_num|age_range_high_num|state|year|data_year|
+-----------+-----------+----------------+------------+-------------------+--------------+------------------+---------------+------+-------+--------+-------+------------+-------------+-------------------------+-------------+--------------+-----------------+------------------+-----+----+---------+
|       1614|     129882|               1|   L55P5MOKP|1991-07-20 00:00:00|             1|                

In [95]:
res = arrestee_AL.select("offense_type_id", "year")

In [132]:
res = arrestee_AL.select("offense_type_id", "year")
off_types = res.groupby(res.offense_type_id, res.year).count()
num_arrests = res.groupby(res.year).count()
num_arrests = num_arrests.select(col("year"), col("count").alias("total_arrests_that_year"))
off_types = off_types.orderBy(desc("count"))
off_types = off_types.select("offense_type_id", "year", col("count").alias("num_arrests"))
#off_types.show()
#num_arrests.show()
off_types = off_types.join(num_arrests, "year", "inner")
#off_types.show()

off_types = off_types.withColumn("perc_tot_arrests", col("num_arrests")/col("total_arrests_that_year"))

In [134]:
shop_lifting = off_types.filter(off_types.offense_type_id == 23)
shop_lifting.show()

+----+---------------+-----------+-----------------------+--------------------+
|year|offense_type_id|num_arrests|total_arrests_that_year|    perc_tot_arrests|
+----+---------------+-----------+-----------------------+--------------------+
|2007|             23|        483|                   1190| 0.40588235294117647|
|2018|             23|          2|                    140|0.014285714285714285|
|2015|             23|        671|                   1413|  0.4748761500353857|
|2006|             23|        525|                   1362|  0.3854625550660793|
|2013|             23|        630|                   1445|  0.4359861591695502|
|2014|             23|        621|                   1270|  0.4889763779527559|
|1991|             23|        789|                   3962| 0.19914184755174155|
|2012|             23|        419|                   1214| 0.34514003294892914|
|2009|             23|        569|                   1441|  0.3948646773074254|
|2016|             23|        113|      

In [136]:
murder = off_types.filter(off_types.offense_type_id == 32)
murder.show()

+----+---------------+-----------+-----------------------+--------------------+
|year|offense_type_id|num_arrests|total_arrests_that_year|    perc_tot_arrests|
+----+---------------+-----------+-----------------------+--------------------+
|2007|             32|          1|                   1190|8.403361344537816E-4|
|2015|             32|          3|                   1413|0.002123142250530...|
|2006|             32|          3|                   1362|0.002202643171806...|
|2013|             32|          1|                   1445|6.920415224913495E-4|
|2014|             32|          1|                   1270|7.874015748031496E-4|
|1991|             32|         57|                   3962|0.014386673397274103|
|2012|             32|          2|                   1214|0.001647446457990...|
|2009|             32|          2|                   1441|0.001387925052047...|
|2016|             32|          2|                    870|0.002298850574712...|
|2005|             32|        280|      

In [137]:
a_assault = off_types.filter(off_types.offense_type_id == 32)
a_assault.show()

+----+---------------+-----------+-----------------------+--------------------+
|year|offense_type_id|num_arrests|total_arrests_that_year|    perc_tot_arrests|
+----+---------------+-----------+-----------------------+--------------------+
|2007|             32|          1|                   1190|8.403361344537816E-4|
|2015|             32|          3|                   1413|0.002123142250530...|
|2006|             32|          3|                   1362|0.002202643171806...|
|2013|             32|          1|                   1445|6.920415224913495E-4|
|2014|             32|          1|                   1270|7.874015748031496E-4|
|1991|             32|         57|                   3962|0.014386673397274103|
|2012|             32|          2|                   1214|0.001647446457990...|
|2009|             32|          2|                   1441|0.001387925052047...|
|2016|             32|          2|                    870|0.002298850574712...|
|2005|             32|        280|      

In [ ]:
victim_circumstances_schema = StructType([ StructField("victim_id", IntegerType(), True)\

                       ,StructField("circumstances_id", IntegerType(), True)\

                       ,StructField("justifiable_force_id", DoubleType(), True)\

                       ,StructField("state", StringType(), True)\

                       ,StructField("data_year", DoubleType(), True)])

In [ ]:
victim_schema = StructType([ StructField("victim_id", IntegerType(), True)\

                       ,StructField("incident_id", IntegerType(), True)\

                       #,StructField("victim_seq_num", IntegerType(), True)\

                       #,StructField("victim_type_id", IntegerType(), True)\

                       #,StructField("assignment_type_id", DoubleType(), True)\

                       #,StructField("activity_type_id", DoubleType(), True)\

                       #,StructField("outside_agency_id", DoubleType(), True)\

                       #,StructField("age_id", DoubleType(), True)\

                       #,StructField("age_num", DoubleType(), True)\

                       #,StructField("sex_code", StringType(), True)\
                       
                       ,StructField("race_id", DoubleType(), True)\
                       
                       ,StructField("ethnicity_id", DoubleType(), True)\
                       
                       #,StructField("resident_status_code", StringType(), True)\
                       
                       #,StructField("agency_data_year", DoubleType(), True)\
                       
                       #,StructField("ff_line_number", DoubleType(), True)\
                       
                       #,StructField("age_range_low_num", DoubleType(), True)\
                       
                       #,StructField("age_range_high_num", DoubleType(), True)\
                       
                       ,StructField("state", StringType(), True)\
                       
                       #,StructField("data_year", DoubleType(), True)\
                       
                       ])

In [138]:
races_schema = StructType([ StructField("race_id", IntegerType(), True)\

                       #,StructField("race_code", StringType(), True)\

                       ,StructField("race_desc", StringType(), True)\

                       #,StructField("start_year", DoubleType(), True)\

                       #,StructField("end_year", DoubleType(), True)\

                       #,StructField("notes", StringType(), True)\

                       #,StructField("state", StringType(), True)

                        ])